In [7]:
import torch
import cv2
import cvzone
import math
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np

# LOAD MODEL

In [2]:
model = YOLO("yolo_weights/yolov8l.pt",)

100%|█████████████████████████████████████████████████████████████████████████████| 83.7M/83.7M [00:07<00:00, 11.9MB/s]


# MAKE DETECTIONS

In [12]:
img = 'https://ultralytics.com/images/zidane.jpg'
results = model("zidane.jpg",show=True)


image 1/1 D:\AI Projects\Detection\6. Drowsiness and Awake Detection\zidane.jpg: 384x640 2 persons, 1 tie, 271.5ms
Speed: 3.0ms preprocess, 271.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


# Real Time Detections

In [20]:
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

model = YOLO("yolo_weights/yolov8l.pt")

class_names = model.model.names

while True:
    success,img = cap.read()
    results = model(img,stream=True)
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            w,h = x2-x1 , y2-y1
            
            conf = math.ceil((box.conf[0]*100))/100
            
            classes = int(box.cls[0])
            
            currentClass = class_names[classes]
            
            if conf > 0.5:
                cvzone.cornerRect(img,(x1,y1,w,h),l=8,t=2,rt=2,colorR=(255,0,255))
                cvzone.putTextRect(img,f"{currentClass}{conf:0.4f}",(max(0,x1),max(40,y1)),scale=1,thickness=1,offset=5)
    cv2.imshow("image",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 2 bottles, 3 beds, 1 tv, 46.5ms
Speed: 1.0ms preprocess, 46.5ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 2 bottles, 2 beds, 1 tv, 49.8ms
Speed: 0.0ms preprocess, 49.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 2 bottles, 1 couch, 3 beds, 1 tv, 50.7ms
Speed: 1.0ms preprocess, 50.7ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 2 bottles, 2 beds, 1 tv, 46.9ms
Speed: 1.0ms preprocess, 46.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 2 bottles, 2 couchs, 2 beds, 1 tv, 33.7ms
Speed: 0.0ms preprocess, 33.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 2 bottles, 2 beds, 1 tv, 34.4ms
Speed: 1.0ms preprocess, 34.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 2 bottles, 1 couch, 1 bed, 1 tv, 35.3ms
Speed: 1.0ms 

# TRAIN OUR CUSTOM MODEL

In [21]:
import uuid
import os
import time

In [29]:
IMAGES_PATH = os.path.join('data','images')
labels = ['awake','drowsy']
number_img = 20

cap = cv2.VideoCapture(0)
for label in labels:
    print(f"Collecting Images for {label}")
    time.sleep(5)
    
    for img_num in range(number_img):
        print(f"Collecting Images from {label} , image number {img_num}")
        ret,frame = cap.read()
        imagename = os.path.join(IMAGES_PATH , label+"."+f"{str(uuid.uuid1())}.jpg")
        cv2.imwrite(imagename,frame)
        cv2.imshow("Image Collection",frame)
        time.sleep(2)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# LABEL IMAGE INSTALL

In [3]:
#!git clone https://github.com/heartexlabs/labelImg.git
# !pip install pyqt5 lxml --upgrade
# !cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

# Make data.yaml

In [6]:
%%writefile data.yaml

path : ../data
train: images
val: images

nc: 2
names: ["Awake","Drowsy"]

Overwriting data.yaml


# There are total 17 classes including our Awake and Drowsy, these are in 15 and 16th label.
So we need to go to the each and every label and change 15 - 0 , 16 - 1.
Because we have mentioned that that in data.yaml

In [7]:
import glob
files = glob.glob("data/labels/*.txt")
for file in files: 
    with open(file,'r') as f:
        lines = f.readlines()
        for i,line in enumerate(lines):
            words = line.split()
            if words[0]=='15':
                words[0]='0'
                lines[i] = " ".join(words) +"\n"
            elif words[0] == '16':
                words[0]='1'
                lines[i] = " ".join(words) +"\n"
    with open(file,'w') as f:
        f.writelines(lines)

# TRAIN YOLO

In [11]:
!yolo task=detect mode=train model=yolov8l.pt data=data.yaml epochs=50 imgsz=640

New https://pypi.org/project/ultralytics/8.0.91 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.82  Python-3.11.2 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)
yolo\engine\trainer: task=detect, mode=train, model=yolov8l.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

In [1]:
import math
from ultralytics import YOLO
import numpy as np
import cv2
import cvzone


cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

model = YOLO("yolo_weights/best.pt")

class_names = model.model.names
class_names

while True:
    success,img = cap.read()
    results = model(img,stream=True)
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            w,h = x2-x1 , y2-y1
            
            conf = math.ceil((box.conf[0]*100))/100
            
            classes = int(box.cls[0])
            
            currentClass = class_names[classes]
            
            if conf > 0.5:
                cvzone.cornerRect(img,(x1,y1,w,h),l=8,t=2,rt=2,colorR=(255,0,255))
                cvzone.putTextRect(img,f"{currentClass}{conf:0.4f}",(max(0,x1),max(40,y1)),scale=1,thickness=1,offset=5)
    cv2.imshow("image",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 Awake, 97.7ms
Speed: 1.0ms preprocess, 97.7ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 Awake, 38.9ms
Speed: 3.0ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 Awake, 37.9ms
Speed: 3.0ms preprocess, 37.9ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 Awake, 40.9ms
Speed: 1.0ms preprocess, 40.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 Awake, 35.9ms
Speed: 1.0ms preprocess, 35.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 Awake, 35.9ms
Speed: 1.0ms preprocess, 35.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 Awake, 34.9ms
Speed: 1.0ms preprocess, 34.9ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 Awake, 37.9ms
Speed: 1.0ms preprocess, 37.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)


In [16]:
from ultralytics import YOLO

In [18]:
model = YOLO("yolov5lu.pt")
model.fuse()

100%|███████████████████████████████████████████████████████████████████████████████| 102M/102M [00:11<00:00, 9.23MB/s]
YOLOv5l summary (fused): 303 layers, 53193088 parameters, 0 gradients, 135.0 GFLOPs
